# Minimal MNIST example for BNN-PYNQ

In this notebook we'll use the LFC overlay, which has binarized activations and three hidden layers with 1024 binarized neurons, to classify handwritten digits.

## Set up the overlay and network parameters
We start by instantiating the overlay, and checking which networks are available. We then load the parameters for the **mnist** network, and print the classes that this network can recognize.

In [ ]:
import bnn
c = bnn.PynqBNN(network = bnn.NETWORK_LFC)
print("Available parameters for LFC overlay: " + str(bnn.available_params(bnn.NETWORK_LFC)))
c.load_parameters("mnist")
print("Classes for MNIST network: " + str(c.classes))

## Classify a single preprocessed image from the MNIST test set

Let's try one of the images from the MNIST test set. We can load and display the image using PIL.

In [ ]:
from PIL import Image
# load image using PIL and convert to black and white
img_7 = Image.open("7.png").convert("L")
img_7

To simplify I/O, BNN-PYNQ expects images to be delivered in the format that the MNIST dataset uses for distribution. The **image_to_mnist** function can take care of this conversion for us. Once we have the MNIST-formatted data, we can call the **inference** function with the filename to classify.

In [ ]:
with open("7-mnist-formatted", "wb") as fp:
  c.image_to_mnist(img_7, fp)
ret_7 = c.inference("7-mnist-formatted")
ret_7

The classification result was correct, but how does the performance compare when running this network in pure software instead of the FPGA accelerator? We can instantiate another classifier that uses a pure software implementation to compare.

In [ ]:
c_sw = bnn.PynqBNN(network=bnn.NETWORK_LFC,runtime=bnn.RUNTIME_SW)
c_sw.load_parameters("mnist")
ret_sw = c_sw.inference("7-mnist-formatted")
ret_sw

We get a significant speedup by using the accelerator. In fact, if we classify multiple images in one go, the FPGA accelerator will run even faster. We'll verify this in a later experiment.

## Classifying a black-on-white handwritten digit

The following images were created by drawing them in Pinta (a Paint-like image editor) with a regular mouse, then taking a screenshot. 

In [ ]:
img_6 = Image.open("6_bw.png").convert("L")
img_6

In [ ]:
with open("6-mnist-formatted", "wb") as fp:
  # set invert=True since MNIST expects white-on-black
  c.image_to_mnist(img_6, fp, invert=True)
ret_6 = c.inference("6-mnist-formatted")
ret_6

## Does it work with some noise?
The following was also drawn in Pinta, with some random dots (noise) added to see how the classifier copes with this.

In [ ]:
img_5 = Image.open("5.png").convert("L")
img_5

In [ ]:
with open("5-mnist-formatted", "wb") as fp:
  # set invert=True since MNIST expects white-on-back
  c.image_to_mnist(img_5, fp, invert=True)
ret_5 = c.inference("5-mnist-formatted")
ret_5

## Classifying the entire MNIST test set

Let's verify the accuracy (and performance) of this network using the entire MNIST test set. We'll start by downloading and unzipping the MNIST test set.

In [ ]:
!wget -nc http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz; gunzip -f t10k-images-idx3-ubyte.gz
!wget -nc http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz; gunzip -f t10k-labels-idx1-ubyte.gz
!ls *ubyte

Now we can pass the test set images directly to the **inference_multiple** function.

In [ ]:
ret_test = c.inference_multiple("t10k-images-idx3-ubyte")

By passing a large amount of images at once, the accelerator can run much faster, around 150 thousand images per second. But what about the correctness of the returned results? To verify the correctness of the classifications, we'll load the "answer key" and compare it against what was returned by the accelerator.

In [ ]:
import numpy as np
with open("t10k-labels-idx1-ubyte", 'rb') as f:
  ret_test_golden = np.frombuffer(f.read(), np.uint8, offset=8)
np.unique(ret_test_golden == ret_test, return_counts=True)